In [ ]:
%load_ext autoreload
from __future__ import print_function, division

In [ ]:
%autoreload

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter

In [ ]:
DATA_FILEPATH     = '/scratch/mmd/mimic_data/final/grouping_5/all_hourly_data.h5'
RAW_DATA_FILEPATH = '/scratch/mmd/mimic_data/final/nogrouping_5/all_hourly_data.h5'
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out
    
class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [ ]:
%%time
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
data_full_raw  = pd.read_hdf(RAW_DATA_FILEPATH, 'vitals_labs') 
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

In [ ]:
data_full_lvl2.head()

In [ ]:
data_full_raw.head()

In [ ]:
statics.head()

In [ ]:
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

In [ ]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)

lvl2, raw = [df[
    (df.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (df.index.get_level_values('hours_in') < WINDOW_SIZE)
] for df in (data_full_lvl2, data_full_raw)]

raw.columns = raw.columns.droplevel(level=['label', 'LEVEL1', 'LEVEL2'])

train_frac, dev_frac, test_frac = 0.7, 0.1, 0.2
lvl2_subj_idx, raw_subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, raw, Ys)]
lvl2_subjects = set(lvl2_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"
assert lvl2_subjects == set(raw_subj_idx), "Subject ID pools differ!"

np.random.seed(SEED)
subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)
N_train, N_dev, N_test = int(train_frac * N), int(dev_frac * N), int(test_frac * N)
train_subj = subjects[:N_train]
dev_subj   = subjects[N_train:N_train + N_dev]
test_subj  = subjects[N_train+N_dev:]

[(lvl2_train, lvl2_dev, lvl2_test), (raw_train, raw_dev, raw_test), (Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (lvl2, raw, Ys)
]

idx = pd.IndexSlice
lvl2_means, lvl2_stds = lvl2_train.loc[:, idx[:,'mean']].mean(axis=0), lvl2_train.loc[:, idx[:,'mean']].std(axis=0)
raw_means, raw_stds = raw_train.loc[:, idx[:,'mean']].mean(axis=0), raw_train.loc[:, idx[:,'mean']].std(axis=0)

lvl2_train.loc[:, idx[:,'mean']] = (lvl2_train.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_dev.loc[:, idx[:,'mean']] = (lvl2_dev.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_test.loc[:, idx[:,'mean']] = (lvl2_test.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds

raw_train.loc[:, idx[:,'mean']] = (raw_train.loc[:, idx[:,'mean']] - raw_means)/raw_stds
raw_dev.loc[:, idx[:,'mean']] = (raw_dev.loc[:, idx[:,'mean']] - raw_means)/raw_stds
raw_test.loc[:, idx[:,'mean']] = (raw_test.loc[:, idx[:,'mean']] - raw_means)/raw_stds

In [ ]:
raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test = [
    simple_imputer(df) for df in (raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test)
]
raw_flat_train, raw_flat_dev, raw_flat_test, lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test = [
    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
        raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test
    )
]

for df in lvl2_train, lvl2_dev, lvl2_test, raw_train, raw_dev, raw_test: assert not df.isnull().any().any()

### Task Prediction

#### Hyperparams

In [ ]:
N = 15

LR_dist = DictDist({
    'C': Choice(np.geomspace(1e-3, 1e3, 10000)),
    'penalty': Choice(['l1', 'l2']),
    'solver': Choice(['liblinear', 'lbfgs']),
    'max_iter': Choice([100, 500])
})
np.random.seed(SEED)
LR_hyperparams_list = LR_dist.rvs(N)
for i in range(N):
    if LR_hyperparams_list[i]['solver'] == 'lbfgs': LR_hyperparams_list[i]['penalty'] = 'l2'

RF_dist = DictDist({
    'n_estimators': ss.randint(50, 500),
    'max_depth': ss.randint(2, 10),
    'min_samples_split': ss.randint(2, 75),
    'min_samples_leaf': ss.randint(1, 50),
})
np.random.seed(SEED)
RF_hyperparams_list = RF_dist.rvs(N)

GRU_D_dist = DictDist({
    'cell_size': ss.randint(50, 75),
    'hidden_size': ss.randint(65, 95), 
    'learning_rate': ss.uniform(2e-3, 1e-1),
    'num_epochs': ss.randint(15, 150),
    'patience': ss.randint(3, 7),
    'batch_size': ss.randint(35, 65),
    'early_stop_frac': ss.uniform(0.05, 0.1),
    'seed': ss.randint(1, 10000),
})
np.random.seed(SEED)
GRU_D_hyperparams_list = GRU_D_dist.rvs(N)

In [ ]:
def run_basic(model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, target):
    best_s, best_hyperparams = -np.Inf, None
    for i, hyperparams in enumerate(hyperparams_list):
        print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))
        M = model(**hyperparams)
        M.fit(X_flat_train, Ys_train[target])
        s = roc_auc_score(Ys_dev[target], M.predict_proba(X_flat_dev)[:, 1])
        if s > best_s:
            best_s, best_hyperparams = s, hyperparams
            print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))

    return run_only_final(model, best_hyperparams, X_flat_train, X_flat_dev, X_flat_test, target)

def run_only_final(model, best_hyperparams, X_flat_train, X_flat_dev, X_flat_test, target):
    best_M = model(**best_hyperparams)
    best_M.fit(pd.concat((X_flat_train, X_flat_dev)), pd.concat((Ys_train, Ys_dev))[target])
    y_true  = Ys_test[target]
    y_score = best_M.predict_proba(X_flat_test)[:, 1]
    y_pred  = best_M.predict(X_flat_test)

    auc   = roc_auc_score(y_true, y_score)
    auprc = average_precision_score(y_true, y_score)
    acc   = accuracy_score(y_true, y_pred)
    F1    = f1_score(y_true, y_pred)
    
    return best_M, best_hyperparams, auc, auprc, acc, F1

### Sklearn

In [ ]:
RESULTS_PATH = '/scratch/mmd/extraction_baselines-sklearn.pkl'
with open(RESULTS_PATH, mode='rb') as f: results = pickle.load(f)
    
RERUN = True

In [ ]:
for model_name, model, hyperparams_list in [
    ('RF', RandomForestClassifier, RF_hyperparams_list), ('LR', LogisticRegression, LR_hyperparams_list)
]:
    if model_name not in results: results[model_name] = {}
    for t in ['mort_icu', 'los_3']:
        if t not in results[model_name]: results[model_name][t] = {}
        for n, X_flat_train, X_flat_dev, X_flat_test in (
            ('lvl2', lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test),
            ('raw', raw_flat_train, raw_flat_dev, raw_flat_test)
        ):
            if n in results[model_name][t]:
                print("Finished model %s on target %s with representation %s" % (model_name, t, n))
                if RERUN: 
                    h = results[model_name][t][n][1]
                    results[model_name][t][n] = run_only_final(model, h, X_flat_train, X_flat_dev, X_flat_test, t)
                    
                    print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                    print(results[model_name][t][n][2:])

                    with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)
                continue
                
            print("Running model %s on target %s with representation %s" % (model_name, t, n))
            results[model_name][t][n] = run_basic(
                model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, t
            )
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(results[model_name][t][n][2:])
            with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)

In [ ]:
np.random.seed(SEED+1)
LR_hyperparams_list_2 = LR_dist.rvs(45)
for i in range(45):
    if LR_hyperparams_list_2[i]['solver'] == 'lbfgs': LR_hyperparams_list_2[i]['penalty'] = 'l2'

results_2 = {}
results_2_PATH = '/scratch/mmd/extraction_baselines-sklearn_LR_2_runs.pkl'

for model_name, model, hyperparams_list in [
#     ('RF', RandomForestClassifier, RF_hyperparams_list),
    ('LR', LogisticRegression, LR_hyperparams_list_2)
]:
    if model_name not in results_2: results_2[model_name] = {}
    for t in ['mort_icu', 'los_3']:
        if t not in results_2[model_name]: results_2[model_name][t] = {}
        for n, X_flat_train, X_flat_dev, X_flat_test in (
            ('lvl2', lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test),
#             ('raw', raw_flat_train, raw_flat_dev, raw_flat_test)
        ):
            if n in results_2[model_name][t]:
                print("Finished model %s on target %s with representation %s" % (model_name, t, n))
                if RERUN: 
                    h = results_2[model_name][t][n][1]
                    results_2[model_name][t][n] = run_only_final(model, h, X_flat_train, X_flat_dev, X_flat_test, t)
                    
                    print("Final results_2 for model %s on target %s with representation %s" % (model_name, t, n))
                    print(results_2[model_name][t][n][2:])

                    with open(results_2_PATH, mode='wb') as f: pickle.dump(results_2, f)
                continue
                
            print("Running model %s on target %s with representation %s" % (model_name, t, n))
            results_2[model_name][t][n] = run_basic(
                model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, t
            )
            print("Final results_2 for model %s on target %s with representation %s" % (model_name, t, n))
            print(results_2[model_name][t][n][2:])
            with open(results_2_PATH, mode='wb') as f: pickle.dump(results_2, f)

In [ ]:
for model_name, model, hyperparams_list in [
#     ('RF', RandomForestClassifier, RF_hyperparams_list),
    ('LR', LogisticRegression, LR_hyperparams_list_2)
]:
    if model_name not in results_2: results_2[model_name] = {}
    for t in ['mort_icu', 'los_3']:
        if t not in results_2[model_name]: results_2[model_name][t] = {}
        for n, X_flat_train, X_flat_dev, X_flat_test in (
#             ('lvl2', lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test),
            ('raw', raw_flat_train, raw_flat_dev, raw_flat_test),
        ):
            if n in results_2[model_name][t]:
                print("Finished model %s on target %s with representation %s" % (model_name, t, n))
                if RERUN: 
                    h = results_2[model_name][t][n][1]
                    results_2[model_name][t][n] = run_only_final(model, h, X_flat_train, X_flat_dev, X_flat_test, t)
                    
                    print("Final results_2 for model %s on target %s with representation %s" % (model_name, t, n))
                    print(results_2[model_name][t][n][2:])

                    with open(results_2_PATH, mode='wb') as f: pickle.dump(results_2, f)
                continue
                
            print("Running model %s on target %s with representation %s" % (model_name, t, n))
            results_2[model_name][t][n] = run_basic(
                model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, t
            )
            print("Final results_2 for model %s on target %s with representation %s" % (model_name, t, n))
            print(results_2[model_name][t][n][2:])
            with open(results_2_PATH, mode='wb') as f: pickle.dump(results_2, f)

In [ ]:
for model_name, model, hyperparams_list in [
    ('RF', RandomForestClassifier, RF_hyperparams_list), ('LR', LogisticRegression, LR_hyperparams_list)
]:
    if model_name not in results: results[model_name] = {}
    for t in ['mort_hosp', 'los_7']:
        if t not in results[model_name]: results[model_name][t] = {}
        for n, X_flat_train, X_flat_dev, X_flat_test in (
            ('lvl2', lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test),
            ('raw', raw_flat_train, raw_flat_dev, raw_flat_test)
        ):
            if n in results[model_name][t]:
                print("Finished model %s on target %s with representation %s" % (model_name, t, n))
                if RERUN: 
                    h = results[model_name][t][n][1]
                    results[model_name][t][n] = run_only_final(model, h, X_flat_train, X_flat_dev, X_flat_test, t)
                    
                    print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                    print(results[model_name][t][n][2:])

                    with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)
                continue
                
            print("Running model %s on target %s with representation %s" % (model_name, t, n))
            results[model_name][t][n] = run_basic(
                model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, t
            )
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(results[model_name][t][n][2:])
            with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)